In [1]:
%cd /content
!git clone https://github.com/Tencent-Hunyuan/Hunyuan3D-2.git
%cd /content/Hunyuan3D-2

# 첨부된 api_server.py로 파일 변경 필요

!pip install -r requirements.txt
!pip install -e .

!pip -q install pyngrok
# 깨끗이 종료
!pkill -f "api_server.py" || true
!pkill -f "uvicorn" || true


/content
Cloning into 'Hunyuan3D-2'...
remote: Enumerating objects: 1064, done.
remote: Counting objects: 100% (299/299), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 1064 (delta 232), reused 202 (delta 202), pack-reused 765 (from 1)
Receiving objects: 100% (1064/1064), 81.28 MiB | 40.17 MiB/s, done.
Resolving deltas: 100% (519/519), done.
/content/Hunyuan3D-2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.8/709.8 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 121.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.

In [3]:
# === One-cell launcher: start FastAPI (file-upload endpoint) + open ngrok ===
import os, socket, time, subprocess, requests
from pyngrok import ngrok

# --- 설정 ---
WORKDIR = "/content/Hunyuan3D-2"     # api_server.py 있는 경로
HOST, PORT = "0.0.0.0", 8090
NGROK_TOKEN = # 개인 토큰 필요

# --- 유틸 ---
def wait_for_port(host, port, timeout_sec=600, interval=2):
    start = time.time()
    while time.time() - start < timeout_sec:
        try:
            with socket.create_connection((host, port), timeout=1):
                return True
        except OSError:
            time.sleep(interval)
    return False

# --- 1) 서버 백그라운드 실행 (모델 로딩은 오래 걸릴 수 있음) ---
%cd {WORKDIR}
log_fp = open("server.log", "w")
server = subprocess.Popen(
    ["python", "api_server.py", "--host", HOST, "--port", str(PORT)],
    stdout=log_fp, stderr=subprocess.STDOUT
)

# --- 2) 포트 오픈까지 대기 ---
print("⏳ 서버 기동 대기 중 (모델 로딩 포함)…")
if not wait_for_port("127.0.0.1", PORT, timeout_sec=600, interval=2):
    print("❌ 포트가 열리지 않았습니다. 최근 로그 120줄:")
    try:
        print("\n".join(open("server.log", "r").read().splitlines()[-120:]))
    except Exception as e:
        print("log read error:", e)
    raise SystemExit()

# --- 3) 기존 ngrok 터널 정리 후 1개만 생성 ---
try:
    for t in ngrok.get_tunnels():
        ngrok.disconnect(t.public_url)
except Exception:
    pass
ngrok.kill()

ngrok.set_auth_token(NGROK_TOKEN)
public = ngrok.connect(PORT, "http")
PUBLIC_URL = public.public_url
print("✅ External API URL:", PUBLIC_URL)

# --- 4) (선택) 준비 완료까지 /healthz 폴링 ---
# api_server.py에 healthz가 있다면 사용(없어도 통과됨)
try:
    for _ in range(120):  # 최대 6분
        try:
            r = requests.get(f"{PUBLIC_URL}/healthz", timeout=5)
            js = r.json()
            print("healthz:", js)
            if js.get("status") in ("ready", None):  # 없으면 None으로 바로 진행
                break
        except Exception:
            pass
        time.sleep(3)
except Exception as e:
    print("healthz check skipped:", e)

print("\n🚀 이제 로컬에서 아래 curl로 테스트하세요:")
print(f'curl -X POST "{PUBLIC_URL}/generate" \\')
print('  -F "file=@test.jpg" \\')
print('  -F "octree_resolution=128" -F "num_inference_steps=5" \\')
print('  -F "guidance_scale=5.0" -F "face_count=100000" -F "texture=false" \\')
print('  --output result.glb')


SyntaxError: invalid syntax (ipython-input-3370996120.py, line 8)

In [ ]:
# 최근 로그 120줄 보기
%cd /content/Hunyuan3D-2
print("\n".join(open("server.log").read().splitlines()[-120:]))


/content/Hunyuan3D-2
2025-08-11 01:55:30.078058: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754877330.098309   15039 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754877330.104748   15039 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754877330.120087   15039 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754877330.120122   15039 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754877330.120126   15039 computation_placer.cc:177] c